In [ ]:
from keras.models import load_model
# Load the saved model
loaded_model = load_model('/content/drive/MyDrive/Features_Output/model.h5')

In [ ]:
pip install twilio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.1 MB/s eta 0:00:00


In [ ]:
import os
import librosa
import numpy as np
from keras.models import load_model
from twilio.rest import Client

# Load the saved model
model_path = '/content/drive/MyDrive/Features_Output/model.h5'  # Replace with your model's path
loaded_model = load_model(model_path)

# Twilio credentials
twilio_account_sid = 'AC208186482829d1f914494dfaef46984e'
twilio_auth_token = 'e9472ec22845dc7ef74e3bdb2811a6d0'
twilio_phone_number = '+12058505637'
verified_person_phone_number = '+917729071360'  # Replace with the verified person's phone number

# Initialize Twilio client
client = Client(twilio_account_sid, twilio_auth_token)

# Function to extract and preprocess features
def extract_and_preprocess_features(audio_path, expected_shape):
    try:
        # Load the new audio file
        audio_data, sample_rate = librosa.load(audio_path, sr=None)

        # Extract MFCCs, ZCR, Mel spectrogram, and Chroma features (adjust as needed)
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=20)
        zcr = librosa.feature.zero_crossing_rate(y=audio_data)
        mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
        chroma = librosa.feature.chroma_stft(y=audio_data, sr=sample_rate)

        # Calculate the mean of each feature
        mfcc_mean = np.mean(mfccs, axis=1)
        zcr_mean = np.mean(zcr, axis=1)
        mel_mean = np.mean(mel_spectrogram, axis=1)
        chroma_mean = np.mean(chroma, axis=1)

        # Combine the extracted features into a single feature vector
        combined_features = np.concatenate((mfcc_mean, zcr_mean, mel_mean, chroma_mean))

        # Ensure that the feature vector shape matches the expected shape
        if combined_features.shape[0] < expected_shape[0]:
            # Pad the feature vector with zeros
            padding = np.zeros(expected_shape[0] - combined_features.shape[0])
            combined_features = np.concatenate((combined_features, padding))

        return combined_features
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None

# Function to send alert via Twilio
def send_alert(recipient_phone_number, message):
    message = client.messages.create(
        to=recipient_phone_number,
        from_=twilio_phone_number,
        body=message
    )
    print(f"Alert message sent to {recipient_phone_number}!")

# Replace 'new_audio_path' with the path to your new audio file
new_audio_path = '/content/drive/MyDrive/Animal_Dataset/Tiger/Copy of track 225_augmentedreverb.wav'

# Define the expected input shape of the model
expected_input_shape = (169, )  # Update with your model's expected input shape

# Extract and preprocess features from the new audio file
new_audio_features = extract_and_preprocess_features(new_audio_path, expected_input_shape)

if new_audio_features is not None:
    # Make predictions using the loaded model
    predictions = loaded_model.predict(new_audio_features[np.newaxis, :])

    # Get the predicted class labels (assuming it's a classification problem)
    predicted_class_labels = np.argmax(predictions, axis=1)

    # Create a mapping of class codes to class labels
    class_label_mapping = {
        0: 'Birds',
        1: 'Dog',
        2: 'Elephant',
        3: 'Tiger',
        4: 'Other'
    }

    # Map the class code to class label for the predicted class
    predicted_class_name = class_label_mapping[predicted_class_labels[0]]

    # Check if a wild animal is detected
    if predicted_class_name in ['Birds', 'Dog', 'Elephant', 'Tiger']:
        # Send an alert message to the verified person
        alert_message = f"Animal ({predicted_class_name}) detected! Take necessary action."
        send_alert(verified_person_phone_number, alert_message)
    else:
        print("No such animal detected in the data.")

    # Print the predicted class label for the new audio file
    print(f"Predicted Class Label for {new_audio_path}: {predicted_class_name}")


1/1 [==============================] - 0s 118ms/step
Alert message sent to +917729071360!
Predicted Class Label for /content/drive/MyDrive/Animal_Dataset/Tiger/Copy of track 225_augmentedreverb.wav: Tiger


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
